### In this section, I will involve a medical-domain LLM(Bio-Medical-Llama-3-8B) as an question-generator to make hypothetical questions based on the chunked docments

### Why I choose ContactDoctor/Bio-Medical-Llama-3-8B?

#### Because:

1. It is fine-tuned based on a bounch of clinical documents. So it knows this domian better than general LLM.

2. I tested three models in my test notebook(That's a mess. I will never show it to you). Result is showing Below:

#### Task 1: Generate hypothetical questions for a document and output JSON
* meta-llama/Llama-3.2-1B-Instruct did OK job. very fast.
* meta-llama/Meta-Llama-3-8B-Instruct is out of control, no matter how I prompt it. it run very slowly!
* ContactDoctor/Bio-Medical-Llama-3-8B did amazing job. fast enought.

#### Task 2: Summarize the document
* meta-llama/Llama-3.2-1B-Instruct did OK job. very fast.
* meta-llama/Meta-Llama-3-8B-Instruct is still out of control. very slow!
* ContactDoctor/Bio-Medical-Llama-3-8B did amazing job. fast enought.

#### So I have no any reason not to choose ContactDoctor/Bio-Medical-Llama-3-8B

#### As the model provider's citation:

@misc{ContactDoctor_Bio-Medical-Llama-3-8B, author = ContactDoctor, title = {ContactDoctor-Bio-Medical: A High-Performance Biomedical Language Model}, year = {2024}, howpublished = {https://huggingface.co/ContactDoctor/Bio-Medical-Llama-3-8B}, }

Bio-Medical-Llama-3-8B model is a specialized large language model designed for biomedical applications. It is finetuned from the meta-llama/Meta-Llama-3-8B-Instruct model using a custom dataset containing over 500,000 diverse entries. These entries include a mix of synthetic and manually curated data, ensuring high quality and broad coverage of biomedical topics.

The model is trained to understand and generate text related to various biomedical fields, making it a valuable tool for researchers, clinicians, and other professionals in the biomedical domain.

In [1]:
from mytools import best_dtype, best_device, login_huggingface
from pathlib import Path
import os, re, json, ast
from typing import List
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field

In [2]:
# This function will load the json file to json object
def load_json_list(path: str):    
    with open(path, mode = "r", encoding="utf-8") as f:
        return json.load(f)

In [3]:
workspace_base_path = os.getcwd()
dataset_path = os.path.join(workspace_base_path, "datasets", "chunked_medicine_data.json") 
print(dataset_path)

/home/jovyan/work/datasets/chunked_medicine_data.json


In [24]:
data = load_json_list(dataset_path)

In [5]:
data[:2]

[{'doc_id': '1bf5880b-93ec-4ac9-a0cb-eb35693ccce4',
  'questions': ['why is Phenylephrine prescribed?'],
  'docs': ['phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine'],
  'original_doc': 'phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine'},
 {'doc_id': 'f7ad6ffd-7176-4

In [5]:
login_huggingface()

Login HuggingFace!


### Now I need to use a local LLM to generate hypothetical question for the content.

In [ ]:
#model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
            model_id,
            dtype = best_dtype(),
            device_map={"":best_device()}, 
            low_cpu_mem_usage=True           
        )
# As per my test a few times, I don't think inference this LLM model with quanlization is a good idea. As it did not that good job.
# If I load it fully, it can make amazing job. So I need to comment below line and load it in normal way.
#model = load_model(model_id)  

print("Load tokenizer and base model done!")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Load tokenizer and base model done!


In [14]:
print(model)                    # full architecture tree (long but useful)
print(model.config)             # core hyperparameters (dims, layers, heads…)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [27]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    return_full_text=False,   
    )

# Wrapper normal piple with huggingfacepipeline
hug_pipe = HuggingFacePipeline(pipeline=pipe)

# Transfer the pipeline to Chat mode.
# Because this is a way we can use ChatPromptTemplate to make better prompt.
question_generator = ChatHuggingFace(llm=hug_pipe)

Device set to use cuda


In [28]:
# it is a dynamic prompt which instruct the LLM to response the number of questions based on the length of doc.
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a cautious medical student who generates clinically relevant, self-contained questions. "
     "Ground every question strictly in the provided document; do not invent details or rely on outside knowledge."),
    ("user",
     "Instructions:\n"
     "1) From the document below, write exactly {n} unique QUESTIONS in English only.\n"
     "2) Cover different medical perspectives (aim for breadth), such as: symptoms/signs; diagnosis/differential; "
     "investigations/labs/imaging; treatment/procedures; medications (dose, interactions, adverse effects); "
     "contraindications/precautions; risk factors/prognosis; prevention/patient counseling; special populations "
     "(e.g., pregnancy, breastfeeding, pediatrics, geriatrics).\n"
     "3) Each question must be concise (≤ 25 words), self-contained (avoid pronouns like 'it/this/that'), and "
     "directly supported by the document.\n"
     "4) Do NOT provide any answers or explanations.\n"
     "5) Return ONLY a JSON list that matches exactly this schema: [\"question1\", \"question2\", ..., \"question{n}\"]\n"
     "6) Enclose the JSON list between <json> and </json> tags.\n"
     "7) If the document does not support {n} distinct perspectives, still produce {n} questions but avoid near-duplicates; "
     "prefer covering as many perspectives as possible."),
    ("user", "Document:\n{doc}\n")
])

### Important Note:
1. Structured Outputs with response_format: {type: "json_schema", ...} is only supported with the gpt-4o-mini, gpt-4o-mini-2024-07-18, and gpt-4o-2024-08-06 model snapshots and later.(Reference: https://platform.openai.com/docs/guides/structured-outputs). For now, OpenAI and Claude's LLM support Structured outputs. Huggingface's Model usually output plain text. But we can do CoT or other prompt engineering to guide the model output what we expect.

2. So Never put the whole parser.get_format_instructions()'s structions into the chat_prompt. Just print it out and copy & pasty useful part into the prompt. In this case, I copy & pasty this {"questions": {"items": {"type": "string"}, "title": "Questions", "type": "array"}} to tell LLM this what I need.

3. Never put parser in the chain. We should always use our own regex partern to extract the string we expect, then pass to the parser to do validate, otherwise it will break the process at random point.

### If we don't follow those two rules, we will get totally mess.

In [29]:
def parse_questions(text: str) -> List[str]:
    print(f"Raw return text: {text}")
    if text is None:
        return []

    s = text.strip()
    from_json_tag = False

    # 1) Extract content inside <json> ... </json> (also tolerate </json> and <\json>)
    m = re.search(r"<json>\s*(.*?)\s*<\s*/?\\?json\s*>", s, flags=re.DOTALL | re.IGNORECASE)
    if m:
        s = m.group(1).strip()
        from_json_tag = True

    # 2) Try strict JSON
    try:
        obj = json.loads(s)
        if isinstance(obj, list):
            return [str(x).strip() for x in obj if str(x).strip()]
        if isinstance(obj, str):
            q = obj.strip()
            return [q] if q else []
    except Exception:
        pass

    # 3) JSON-like list embedded in extra text
    m = re.search(r"\[\s*[\s\S]*?\s*\]", s)
    if m:
        candidate = m.group(0)
        try:
            obj = json.loads(candidate)
            if isinstance(obj, list):
                return [str(x).strip() for x in obj if str(x).strip()]
        except Exception:
            try:
                obj = ast.literal_eval(candidate)
                if isinstance(obj, list):
                    return [str(x).strip() for x in obj if str(x).strip()]
            except Exception:
                pass

    # 4) Quoted strings fallback
    rough = re.findall(r'"([^"]+)"', s)
    if rough:
        return [q.strip() for q in rough if q.strip()]

    # 5) If we specifically pulled from <json> tags and it's non-empty text, treat as single-question payload
    if from_json_tag and s:
        return [s]

    # 6) NEW: Plain-text fallback — collect every sentence ending with '?'
    #    Handles single or multiple questions not in quotes/lists/tags.
    qs = [q.strip() for q in re.findall(r'([^?]*\?)', s, flags=re.DOTALL)]
    qs = [q for q in (q.strip() for q in qs) if q]  # clean empties
    if qs:
        # Optionally deduplicate while preserving order
        seen = set()
        uniq = []
        for q in qs:
            if q not in seen:
                seen.add(q)
                uniq.append(q)
        return uniq
    print("Return empty list")
    return []

In [30]:
# Decide how many questions it need to generate based on the length of the document.
# From my experience, 500 to 2000 : 10 questions, 100 to 500 : 5 questions, less than 100 : 1 question.
# But it can be varied.
def decide_n_questions(txt: str):
    wc = len(txt.split(" "))    
    if wc > 500:
        return 10
    elif wc > 100:
        return 5
    else:
        return 1

In [31]:
# If question_generator already returns text:
full_chain = (prompt | question_generator | StrOutputParser() | RunnableLambda(parse_questions))

In [ ]:
# Now let's iterate all medicine's docments and invoke chain to automatically generate question. 

for content in data[:500]:
    new_questions = []
    for doc in content["docs"]:
        n = decide_n_questions(doc)   
        res = full_chain.invoke({"n": n, "doc": doc})        
        if len(res) > 0:
            new_questions.extend(res)
    content['questions'].extend(new_questions)

In [23]:
data[:5]

[{'doc_id': '1bf5880b-93ec-4ac9-a0cb-eb35693ccce4',
  'questions': ['why is Phenylephrine prescribed?',
   'What are the typical uses for phenylephrine?',
   'Can phenylephrine be used to treat the cause of nasal congestion?',
   'Can phenylephrine be used to relieve nasal discomfort caused by common cold?'],
  'docs': ['phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine'],
  'original_doc': 'phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed

In [ ]:
save_to = os.path.join(workspace_base_path, "datasets", "medicine_data_hypotheticalquestions2.json") 

out_path = Path(save_to)
out_path.parent.mkdir(parents=True, exist_ok=True)  # make folder if needed

with out_path.open("w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)